In [37]:
""" Reads json files from Megadetector output and produces cropped images with bounding boxes for each animal found """

' Reads json files from Megadetector output and produces cropped images with bounding boxes for each animal found '

In [1]:
import json
import cv2
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# read json output for given batch
def read_file(path, batch):
    with open(f"{path}/output_{batch}.json") as json_file:
        data = json.load(json_file)
    return data

In [3]:
# obtain relevant detections from the data
def get_detections(data):

    # lists to store images where megadetector confidently recognized animals
    # and the detection information that it captured
    img_names = []
    img_detection = []

    for img in data["images"]:
        file_name = img["file"]
        detections = img["detections"]
        if len(detections) < 1:
            continue
        
        for detection in detections:
            # skip if the detector is not at least 50% confident
            if detection["conf"] < 0.5:
                continue
            
            # skip if it's not an animal
            if detection["category"] != "1":
                continue
            
            else:
                img_names.append(file_name)
                img_detection.append(detection)
                
    return (img_names, img_detection)

In [4]:
# add rectangle with given bounding box coordinates to given image
def add_bbox(img, box):
    y_dim, x_dim, _ = img.shape
    
    # coordinates are given in relative terms so need to convert them to absolute values
    x0 = round(box[0] * x_dim)
    x1 = x0 + round(box[2] * x_dim)
    y0 = round(box[1] * y_dim)
    y1 = y0 + round(box[3] * y_dim)
    
    # (x0, y0) are coordinates of the the top left corner of bounding box
    # (x0+x1, y0+y1) are the coordinates of the bottom right corner of bounding box

    start_point = (x0, y0)
    end_point = (x1, y1)

    return cv2.rectangle(img, start_point, end_point, color=(255,0,0), thickness=2)

In [5]:
# crop given image based on bounding box coordinatees
def crop_box(img, box):
    y_dim, x_dim, _ = img.shape
    
    # coordinates are given in relative terms so need to convert them to absolute values
    x0 = round(box[0] * x_dim)
    x1 = x0 + round(box[2] * x_dim)
    y0 = round(box[1] * y_dim)
    y1 = y0 + round(box[3] * y_dim)
    
    return img[y0:y1, x0:x1, :]

In [6]:
# write images with bounding boxes to given path
# and return list with modified images
def write_bb_img(img_names, img_detections, batch, path):
    
    for name, det in zip(img_names, img_detections):
        # img_name = f"images/loc-h500ee05127658/{batch}/{name}"
        img_name = f"images/jldp/{batch}/prod/directUpload/{name}"
        img = cv2.imread(img_name)
        box = det["bbox"]
        
        # img_bbox = add_bbox(img, box)
        img_bbox = crop_box(img, box)
    
        # new_name = f"img-{batch}-{name}"
        new_name = name
        full_path = f"{path}/{new_name}"
        
        cv2.imwrite(full_path, img_bbox)

In [38]:
# to do: 2139507, 2139509, 2139571
batches = ["2252500"]
path = "megadetector-output-jldp"
for batch in batches:
    data = read_file(path, batch)
    img_names, img_detections = get_detections(data)
    write_bb_img(img_names, img_detections, batch, "images-bboxes/jldp")

In [36]:
# plt.imshow(img)